<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Правило Симпсона и метод трапеций

### Modules - Numerical Integration
<section class="post-meta">
By Magnus A. Gjennestad, Vegard Hagen, Aksel Kvaal, Morten Vassvik, Trygve B. Wiig and Peter Berg
</section>
Last edited: March 11th 2018 
___

### Задача

Как решить определенный интеграл, такой как
$$\int_a^b f(x) \mathrm{d}x$$
если мы не можем решить его аналитически, т.е. если мы не можем найти функцию $F(x)$ такую, что
$$\frac{\mathrm{d}}{\mathrm{d}x}F(x) = f(x)$$
и 
$$ \int_a^b f(x) \mathrm{d}x = F(b)-F(a).$$

### Пример

Мы не можем взять интеграл
$$ \int_0^1 \sqrt{x^5+e^{5x}} \mathrm{d}x$$
аналитически. Так что же нам делать?

Построим подынтегральное выражение $f(x)=\sqrt{x^5+e^{5x}}$ для $x\in[0,1]$. Сначала мы импортируем необходимые библиотеки и задаем некоторые параметры для построения графиков:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Устанавливаем общие параметры
newparams = {'figure.figsize': (16, 6), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 14}
plt.rcParams.update(newparams)

In [ ]:
x = np.linspace(0,1,100)
def f(x): return np.sqrt(x**5+np.exp(5*x))

plt.plot(x, f(x))
plt.ylabel(r'$f(x)$')
plt.xlabel(r'$x$');

Мы знаем, что интеграл представляет в данном случае область под графиком. Область хорошо определена и конечна.

Мы могли бы аппроксимировать его суммой Римана. Для этого мы разделяем интервал $[0, 1]$ на $N − 1$ интервалов одинаковой длины $h$ и
$$h = \frac{1}{N-1}.$$

Конечные точки этих интервалов расположены по адресу
$$x_n = (n-1)h,\qquad \mathrm{with}\quad n=1,2,3,...,N.$$
Это естественным образом приводит к определению прямоугольника для каждого интервала $[x_n, x_{n+1}]$, высота которого определяется значением функции в $x_n$, т.е. $f(x_n)$.

Для N = 6 мы можем нарисовать это на нашем предыдущем графике. Но сначала нам нужно определить функцию для рисования трапеций:

In [ ]:
from matplotlib.path import Path
import matplotlib.patches as patches

def draw_trapezoid(xpoints, ypoints):
    """ Функция для рисования трапеций. Принимает массивы значений x и y в качестве входных данных,
перемещаясь по часовой стрелке от нижнего левого угла. """
    verts = [
        (xpoints[0], ypoints[0]), # left, bottom
        (xpoints[1], ypoints[1]), # left, top
        (xpoints[2], ypoints[2]), # right, top
        (xpoints[3], ypoints[3]), # right, bottom
        (0., 0.), # ignored
        ]

    codes = [Path.MOVETO,
             Path.LINETO,
             Path.LINETO,
             Path.LINETO,
             Path.CLOSEPOLY,
             ]

    path = Path(verts, codes)
    ax = plt.gca()
    patch = patches.PathPatch(path, facecolor='#d3d3d3', lw=1)
    ax.add_patch(patch)

In [ ]:
plt.plot(x, f(x))
plt.ylabel(r'$f(x)$')
plt.xlabel(r'$x$')

# Рисует прямоугольники
N = 6
h = 1/(N-1)
for n in range(N):
    draw_trapezoid([(n-1)*h,(n-1)*h,n*h,n*h], [0,f((n-1)*h),f((n-1)*h),0])

Мы видим, что мы допускаем довольно большую ошибку в нашей оценке интеграла, когда используем эти прямоугольники. Как  сделать лучше?

### Метод трапеций

Один из способов продвинуться вперед - сделать интервалы меньше. В пределе, когда их длина стремится к нулю, мы должны найти точное значение интеграла.

Однако это непрактично для компьютера, поскольку требует бесконечного количества вычислений. Что еще мы можем сделать?

Что ж, лучшее приближение для нашего последнего набора интервалов можно получить, если вместо этого мы используем трапеции. Две стороны (верхняя и нижняя) каждой трапеции определяются интервалом вдоль оси x, в то время как две другие стороны (левая и правая) определяются значениями функции $f(x)$ на каждом конце интервала.
Давайте посмотрим, как это выглядит.

In [ ]:
plt.plot(x, f(x))
plt.ylabel(r'$f(x)$')
plt.xlabel(r'$x$')

# Plot trapezoids:
N = 6
h = 1/(N-1)
for n in range(N):
    draw_trapezoid([(n-1)*h,(n-1)*h,n*h,n*h], [0,f((n-1)*h),f((n)*h),0])

Похоже, что мы совершаем гораздо меньшую ошибку, чем раньше, учитывая тот же набор интервалов. Ошибка, по-видимому, увеличивается, когда увеличивается кривизна графика (справа на этом графике).

Выведем формулу для площади, которую покрывают трапеции. Используя наши обозначения выше, площадь трапеции задается
$$A_n=h\frac{f_n+f_{n+1}}{2} $$
где $f_n=f(x_n)$.
Здесь $h$ - это длина каждого интервала и, следовательно, длина основания каждой трапеции, а выражение
$$\frac{f_n + f_{n+1}}{2}$$
представляет "среднюю высоту" соответствующей трапеции.

Сложив все трапеции, получим общую площадь
$$A=A_1 +A_2 +A_3 +...+A_{N−1}$$
и вот
$$A=h\frac{f_1 +f_2}{2} +h\frac{f_2 +f_3}{2} +h\frac{f_3 +f_4}{2} +...h\frac{f_{N−1} +f_N}{2}.$$
Разложение на множители h и перестановка слагаемых дает
$$A=h\left(\frac{1}{2}f_1+f_2+f_3+...+f_{N−1}+\frac{1}{2}f_N\right).$$
Эта формула описывает __метод трапеций__ и аппроксимирует исходный интеграл.

Слагаемые внутри скобок будут равны: 0.5, 1, 1, ..., 1, 0.5. Это связано с тем, что мы используем конечные точки 0 и 1 только один раз в наших расчетах, в то время как все остальные точки дважды участвуют в определении трапециевидных областей.

Обратите внимание, что нам не нужно предполагать интервал $[0, 1]$. Вместо этого мы можем использовать этот метод для любого интервала $[a, b]$. Затем определения изменяются 
$$h= \frac{b−a}{N-1}\qquad \mathrm{and} \qquad x_n =a+(n−1)h.$$
Таким образом, мы можем аппроксимировать интеграл
$$\boxed{\int_a^b f(x) \mathrm{d}x \approx h\left(\frac{1}{2}f_1+f_2+f_3+...+f_{N−1}+\frac{1}{2}f_N\right).}$$

Можно показать, какую ошибку имеет этот метод (т.е. фактическое значение определенного интеграла по сравнению с числовым значением), которая пропорциональна

1. $(b−a)^3$,
2. $\frac{1}{N^2}$, и
3. (максимальной) величине второй производной $f(x)$
на соответствующем интервале.

Следовательно, удвоение количества интервалов уменьшает ошибку в четыре раза! Однако выбор очень больших $N$ не является хорошей идеей по двум причинам:
- Это увеличивает вычислительное время.
- Компьютер обладает конечной точностью; ошибки округления становятся важными!

Вторая производная связана с кривизной графика, и мы видели выше, что ошибка действительно увеличивалась там, где кривизна была больше. Это имеет смысл, поскольку мы пытаемся аппроксимировать криволинейный график прямой линией.

Давайте приблизим интеграл
$$ \int_0^1 \sqrt{x^5+e^{5x}} \mathrm{d}x$$
для различных $N$. По мере увеличения N значения трапециевидного правила сходятся. Попробуйте сами!

In [ ]:
# Определяет интервал
a = 0.0
b = 1.0

N = 10                  # определение количества точек, составляющих меньшие интервалы
n = np.linspace(1, N, N)
h = (b-a)/(N-1)         # определение длины каждого небольшого интервала

area = 0                # установка области на ноль

for i in n:
    x = a + (i-1)*h
    func = np.sqrt(x**5+np.exp(5*x))
    if (i==1 or i==N):  # Первый и последний пункт
        area += 0.5*func*h
    else:
        area += func*h
        
print(r'Значение интеграла равно %f.' % area)

## Правило Симпсона

__Вопрос:__
Существует ли лучший, т. е. более точный метод аппроксимации интеграла? Можем ли мы сделать то же самое с меньшими интервалами?

__Ответ:__
Да, есть несколько методов.
Один из методов называется __правилом Симпсона__. Идея этого метода заключается в том, что мы аппроксимируем функцию локально не прямой линией, как в случае трапециевидного метода, а полиномом порядка 2. Это означает, что мы аппроксимируем функцию $f(x)$ в каждом подинтервале
$$f(x) \approx c_n x^2 + d_n x + e_n =: g_n(x).$$
Параметры $c_n$, $d_n$, $e_n$ выбираются таким образом, чтобы $f(x)$ и $g_n(x)$ совпадали в конечных точках интервала и в его середине.

#### Пример:

Построим график нашей исходной функции и аппроксимируем ее полиномом второго порядка с вышеуказанными характеристиками. Мы получаем уравнения
\begin{align*}
e_n &= f(0), \\
\frac{1}{4}c_n+\frac{1}{2}d_n+e_n &= f(0.5), \\
c_n+d_n+e_n &= f(1),
\end{align*}
что дает
\begin{align*}
c_n &= 2f(1)-4f(0.5)+2f(0),\\
d_n &= -f(1) + 4f(0.5) - 3f(0),\\
e_n &= f(0).
\end{align*}

In [ ]:
# Определяет значения параметров:
c_n = 2*(f(1)-2*f(0.5)+f(0))
d_n = -f(1)+4*f(0.5)-3*f(0)
e_n = f(0)

x = np.linspace(0,1,100)
def g(x): return c_n*x**2 + d_n*x + e_n

plt.plot(x, f(x), x, g(x))
plt.ylabel(r'$f(x)$')
plt.xlabel(r'$x$')
plt.legend([r'$f(x)$', r'$g(x)$'], loc=2);

Мы видим, что полином уже достаточно хорошо описывает исходную функцию. А мы даже не начинали делить интервал $[0, 1]$ на подинтервалы...

Важно отметить, что правило Симпсона требует и нечетного количества точек ($N$ = 3, 5, 7, ...) вдоль оси $x$, так как нам нужно использовать три точки для каждого интервала, т.е. левую конечную точку, правую конечную точку и среднюю точку.

В то же время два соседних интервала имеют одну конечную точку, т. е. правая конечная точка одного интервала совпадает с левой конечной точкой другого интервала (за исключением $x = b$).

Все это время мы по-прежнему сохраняем равномерное деление исходного интервала $[a, b]$ на более мелкие подинтервалы.

Простейший случай - использование трех точек для исходного интервала, представляющих один интервал с одной средней точкой:
$$x_1 =a,\, x_2 =\frac{b-a}{2},\, x_3 =b.$$
Этот случай показан в предыдущем сюжете. В этом случае можно показать, что интегральное приближение сводится к
$$\int_a^b f(x) \mathrm{d}x \approx \frac{b-a}{6}\left[f(x_1)+4f(x_2)+f(x_3)\right].$$
В следующем случае используется пять точек: два интервала с одной средней точкой в каждом. Теперь мы получаем
$$\int_a^b f(x) \mathrm{d}x \approx \frac{h}{3}\left[f(x_1)+4f(x_2)+2f(x_3)+4f(x_4)+f(x_5)\right],$$
где $h$-расстояние между двумя последовательными точками вдоль оси $x$. Этот сценарий показан ниже.

In [ ]:
N = 5
h = (b-a)/(N-1)
# Plot lines
color = ['r', 'g', 'r', 'g', 'r']
for i in range(N):
    plt.plot([i*h, i*h],[0, f(i*h)], color[i], lw=2)

plt.plot(x, f(x),lw=2)
plt.ylabel(r'$f(x)$')
plt.xlabel(r'$x$');

Случай двух интервалов: $[0, 0.5]$ и $[0.5, 1]$. У нас есть в общей сложности 5 точек вдоль оси $x$, а именно 3 конечные точки интервала $(x_1 = 0,\, x_3 = 0.5,\, x_5 = 1)$ и 2 средние точки
$(x_2 = 0,25,\, x_4 = 0,75)$. Правило Симпсона использует полиномы второго порядка в каждом интервале.

Общее выражение для произвольного количества интервалов имеет вид
$$\int_a^b f(x) \mathrm{d}x \approx \frac{h}{3}\left[f(x_1)+4f(x_2)+2f(x_3)+4f(x_4)+...2f(x_{N-2})+4f(x_{N-1})+f(x_N)\right],$$
или используя нашу предыдущую нотацию выше
$$\boxed{\int_a^b f(x) \mathrm{d}x \approx \frac{h}{3}\left[f_1+4f_2+2f_3+4f_4+...+2f_{N-2}+4f_{N-1}+f_N\right].}$$
Давайте вычислим тот же интеграл, что и для трапециевидного правила, только на этот раз используя правило Симпсона.

In [ ]:
a = 0.0           # определение интервала
b = 1.0           
N = 3             # определение количества точек, составляющих меньшие интервалы
n = np.linspace(1, N, N)
h = (b-a)/(N-1)   # определение длины каждого небольшого интервала
area = 0          # установка area на ноль
 
for i in n:
    x = a+(i-1)*h        
    func = np.sqrt(x**5+np.exp(5*x))
    if (i==1) | (i==N):    # если мы находимся в левом или правом конце главного
        area += func         # интервала
    else:
        if i%2==0:         # если мы находимся в "четной" точке
            area += 4*func     
        else:              # если мы находимся в "нечетной" точке
            area += 2*func
 
area = (h/3)*area    # умножение всего на h/3

print(r'Значение интеграла равно %f.' % area)

Попробуйте для разных $N$. Сходится ли оно быстрее, чем раньше?

В отличие от трапециевидного правила, правило Симпсона содержит ошибку, пропорциональную
1. $(b−a)^5$,
2. $\frac{1}{N^4},$ and
3. (максимальной) величине четвертой производной $f(x)$ на соответствующем интервале.

Другими словами, он превосходит по точности трапециевидное правило относительно количества интервалов $N$.